# Graph partitioning

The goal is to partition the nodes of a graph to sets of equal size with minimum number of edges between the sets.

In [ ]:
from dimod import DiscreteQuadraticModel, ExactDQMSolver
from itertools import combinations
import networkx as nx

# Graph partitioning with more than 2 sets

In [ ]:
edges = [
    (1, 2), (2, 3), (1, 3), 
    (4, 5), (5, 6), (4, 6), 
    (7, 8), (8, 9), (7, 9),
    (2, 4), (4, 8)
]
nodes = sorted(set().union(*edges))
graph = nx.Graph(edges)
nx.draw(graph, with_labels=True)

# Decision variable
This time we assign a binary variable $x_{i,k}$ for each node $i$ and set $k$. If the value is one, the node $i$ belongs to set $k$.

In [ ]:
dqm = DiscreteQuadraticModel()

m = 3
n = len(nodes)

for node in nodes:
    dqm.add_variable(m, node)

# Objective
Rather than penalizing two nodes that are not in the same set, we reward if
- two nodes belong to the same set, and
- the two nodes are connected

In [ ]:
for a, b in edges:
    for k in range(m):
        dqm.set_quadratic_case(a, k, b, k, -1)   

# Constraints

The size of each set is the n/m
- n is the number of nodes
- m is the number of sets

In [ ]:
for k in range(m):
    dqm.add_linear_equality_constraint(
        [(i, k, 1.0) for i in nodes],
        constant=-n/m,
        lagrange_multiplier=10
    )    

Each node can only belong to one set. This is naturally handled by the DQM object and all DQM solvers.

In [ ]:
res = ExactDQMSolver().sample_dqm(dqm).truncate(10)
print(res)

In [ ]:
sample = res.first.sample
sample

In [ ]:
colors = ['r', 'g', 'b']
node_color = [colors[res.first.sample[node]] for node in nodes]
nx.draw(graph, with_labels=True, node_color=node_color, font_color='w')